In [14]:
UTILISATEUR = {
    "Moussa" :  {"USER" : "moussa.dieng@atmosud.org", "password" : "XM:~G'%SL>26pr2"}
}

In [ ]:
import json
import requests
import os
from getpass import getpass
from tqdm import tqdm
#import zipfile
from datetime import datetime
import re

# Définir les paramètres personnalisés
POLLUANT = "AER_AI"  #  "O3"   #"NO2"
ZONE = "POLYGON((4.15 43.01, 7.73 43.01, 7.73 44.76, 4.15 44.76, 4.15 43.01))"
START_DATE = "2024-03-01T00:00:00.000Z"
END_DATE = "2024-03-01T23:59:59.999Z"

# Définir les informations d'utilisateur

USERNAME = UTILISATEUR["Moussa"]["USER"]
password = UTILISATEUR["Moussa"]["password"]

#USERNAME = "moussa.dieng@atmosud.org"
#password = "XM:~G'%SL>26pr2"      # mot de passs compte COPERNICUS password = "XM:~G'%SL>26pr2"
def get_keycloak(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password"
    }
    r = requests.post("https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token", data=data)
    r.raise_for_status()
    return r.json()["access_token"], r.json()["refresh_token"]

def refresh_keycloak(refresh_token: str) -> str:
    data = {
        "client_id": "cdse-public",
        "grant_type": "refresh_token",
        "refresh_token": refresh_token
    }
    r = requests.post("https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token", data=data)
    r.raise_for_status()
    return r.json()["access_token"], r.json()["refresh_token"]

#passwd = str(getpass())
passwd = password 
keycloak_token, refresh_token = get_keycloak(USERNAME, passwd)

def download_zip(url: str, fname: str, chunk_size=1024):
    session = requests.Session()
    session.headers.update({'Authorization': 'Bearer {}'.format(keycloak_token)})

    resp = requests.get(url, allow_redirects=False)
    while resp.status_code in (301, 302, 303, 307):
        url = resp.headers['Location']
        resp = session.get(url, verify=True, stream=True, allow_redirects=False)
    
    total = int(resp.headers.get('content-length', 0))
    with open(fname, 'wb') as file, tqdm(desc=fname, total=total, unit='iB', unit_scale=True, unit_divisor=1024) as bar:
        for data in resp.iter_content(chunk_size=chunk_size):
            size = file.write(data)
            bar.update(size)

print("Recherche des produits...")

if POLLUANT == "AER_AI":
   products_url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?&$filter=((Collection/Name%20eq%20%27SENTINEL-5P%27%20and%20(Attributes/OData.CSC.StringAttribute/any(att:att/Name%20eq%20%27instrumentShortName%27%20and%20att/OData.CSC.StringAttribute/Value%20eq%20%27TROPOMI%27)%20and%20(contains(Name,%27L2__{POLLUANT}_%27)%20and%20OData.CSC.Intersects(area=geography%27SRID=4326;{ZONE}%27)))%20and%20Online%20eq%20true)%20and%20ContentDate/Start%20ge%20{START_DATE}%20and%20ContentDate/Start%20lt%20{END_DATE})&$orderby=ContentDate/Start%20desc&$expand=Attributes&$count=True&$top=1000&$expand=Assets&$skip=0"
else :
   products_url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?&$filter=((Collection/Name%20eq%20%27SENTINEL-5P%27%20and%20(Attributes/OData.CSC.StringAttribute/any(att:att/Name%20eq%20%27instrumentShortName%27%20and%20att/OData.CSC.StringAttribute/Value%20eq%20%27TROPOMI%27)%20and%20(contains(Name,%27L2__{POLLUTANT}___%27)%20and%20OData.CSC.Intersects(area=geography%27SRID=4326;{ZONE}%27)))%20and%20Online%20eq%20true)%20and%20ContentDate/Start%20ge%20{START_DATE}%20and%20ContentDate/Start%20lt%20{END_DATE})&$orderby=ContentDate/Start%20desc&$expand=Attributes&$count=True&$top=1000&$expand=Assets&$skip=0"

session = requests.Session()
session.headers.update({'Authorization': 'Bearer {}'.format(keycloak_token)})
response = requests.get(products_url, headers=session.headers)

try:
    lines = response.json()
except json.JSONDecodeError:
    print("Erreur : Impossible de décoder la réponse JSON.")
    exit(1)

if "value" not in lines:
    print("Erreur : La clé 'value' est absente de la réponse API.")
    print("Réponse API :", lines)
    exit(1)

# Chemin de stockage automatiquement en fonction du polluant
destination_root =f"N:/MOD_SERVER/SATELLITES/tropomi_s5_annuel/alternance_moussa/output/{POLLUTANT}"
if not os.path.exists(destination_root):
    os.makedirs(destination_root)

for value_data in lines["value"]:
    product_name = value_data["Name"]
    product_identyficator = str(value_data['Id'])

    match = re.search(r"(\d{8}T\d{6})", product_name)
    if not match:
        print(f"Erreur : Impossible d'extraire la date du fichier {product_name}")
        continue

    date_str = match.group(1)
    date_obj = datetime.strptime(date_str, "%Y%m%dT%H%M%S")
    folder_name = f"{date_obj.year}_{date_obj.month:02d}"
    destination_folder = os.path.join(destination_root, folder_name)

    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    file_path = os.path.join(destination_folder, product_name)

    url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products({product_identyficator})/$value"
    keycloak_token, refresh_token = refresh_keycloak(refresh_token)

    download_zip(url, file_path)

    print(f"Téléchargement terminé : {file_path}")



Recherche des produits...


N:/MOD_SERVER/SATELLITES/tropomi_s5_annuel/alternance_moussa/output/AER_AI\2024_03\S5P_OFFL_L2__AER_AI_20240301T122838_20240301T141009_33073_03_020600_20240303T021827.nc: 100%|██████████| 198M/198M [02:27<00:00, 1.41MiB/s]    


Téléchargement terminé : N:/MOD_SERVER/SATELLITES/tropomi_s5_annuel/alternance_moussa/output/AER_AI\2024_03\S5P_OFFL_L2__AER_AI_20240301T122838_20240301T141009_33073_03_020600_20240303T021827.nc


N:/MOD_SERVER/SATELLITES/tropomi_s5_annuel/alternance_moussa/output/AER_AI\2024_03\S5P_OFFL_L2__AER_AI_20240301T104708_20240301T122838_33072_03_020600_20240303T003853.nc: 100%|██████████| 212M/212M [02:41<00:00, 1.38MiB/s]   

Téléchargement terminé : N:/MOD_SERVER/SATELLITES/tropomi_s5_annuel/alternance_moussa/output/AER_AI\2024_03\S5P_OFFL_L2__AER_AI_20240301T104708_20240301T122838_33072_03_020600_20240303T003853.nc


### **Traitement des données et Rasters moyens Journaliers, mensuels ou annuels**

In [ ]:
import os
import numpy as np
import rasterio
import xarray as xr
import pandas as pd
from shapely.geometry import Polygon, Point
import geopandas as gpd
from scipy.spatial import cKDTree
from rasterio.transform import from_origin


def create_parallelogram_grid(lat_min, lat_max, lon_min, lon_max, dx, dy):
    """Génère une grille de parallélogrammes couvrant la région spécifiée."""
    polygons, centers = [], []

    for x in np.arange(lon_min, lon_max, dx):
        for y in np.arange(lat_min, lat_max, dy):
            p = Polygon([
                (x, y), (x + dx, y), (x + dx + dy / 2, y + dy), (x + dy / 2, y + dy)
            ])
            polygons.append(p)
            centers.append(Point(x + dx / 2, y + dy / 2))

    return gpd.GeoDataFrame(geometry=polygons), centers

def process_netcdf_to_raster(year: str, pollutant: str, base_dir: str, qa_min: float = 0.75, month: str = None, day: str = None):
    """
    Génère un raster journalier, mensuel ou annuel en combinant les fichiers NetCDF.

    :param year: Année 
    :param pollutant: Polluant à traiter (ex: "NO2")
    :param base_dir: Répertoire de base contenant les fichiers NetCDF
    :param month: Mois (ex: "07") ou None pour l'année entière
    :param day: Jour (ex: "02") ou None pour le mois entier
    """
    pollutant = pollutant.upper()
    if pollutant not in POLLUANTS_CONFIG:
        print(f" Polluant '{pollutant}' non reconnu.")
        return

    var_name = POLLUANTS_CONFIG[pollutant]["var"]
    periods = [f"{year}_{month}"] if month else [f"{year}_{m:02d}" for m in range(1, 13)]
    
    # Déterminer le fichier de sortie
    if day:
        output_name = f"{pollutant}_mean_{year}_{month}_{day}.tif"
        print(f" Génération du raster journalier ({day}/{month}/{year})")
    elif month:
        output_name = f"{pollutant}_mean_{year}_{month}.tif"
        print(f" Génération du raster mensuel ({month}/{year})")
    else:
        output_name = f"{pollutant}_mean_{year}.tif"
        print(f" Génération du raster annuel ({year})")

    # Définition de la grille
    lat_min, lat_max = 42.3, 45.4
    lon_min, lon_max = 3.63, 8
    resolution_x, resolution_y = 0.1, 0.06
    grid, grid_centers = create_parallelogram_grid(lat_min, lat_max, lon_min, lon_max, resolution_x, resolution_y)

    # Initialisation des matrices
    sum_values = np.zeros(len(grid), dtype=np.float32)
    count_values = np.zeros(len(grid), dtype=np.int32)

    # Recherche des fichiers NetCDF
    files = []
    for period in periods:
        input_dir = os.path.join(base_dir, pollutant, period)
        if not os.path.exists(input_dir):
            continue
        files.extend([os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(".nc")])

    if not files:
        print(f" Aucun fichier NetCDF trouvé pour {pollutant} en {year} ({month if month else 'Année entière'})")
        return

    for file in files:
        if day and day not in file:
            continue

        ds = xr.open_dataset(file, group="PRODUCT")
        data_array = ds[var_name].isel(time=0).values.astype(np.float32)
        qa = ds["qa_value"].isel(time=0).values.astype(np.float32)
        lat = ds["latitude"].values.astype(np.float32)
        lon = ds["longitude"].values.astype(np.float32)
        ds.close()

        df = pd.DataFrame({
            "lat": lat.ravel(),
            "lon": lon.ravel(),
            "value": data_array.ravel(),
            "qa_value": qa.ravel()
        })
        df = df[df["qa_value"] >=qa_min]
        df = df[(df["lat"] >= lat_min) & (df["lat"] <= lat_max) & 
                (df["lon"] >= lon_min) & (df["lon"] <= lon_max)]

        if df.empty:
            continue

        tree = cKDTree(np.array([[p.x, p.y] for p in grid_centers]))
        indices = tree.query(df[['lon', 'lat']].values)[1]

        for i, idx in enumerate(indices):
            sum_values[idx] += df['value'].iloc[i]
            count_values[idx] += 1

    # Calcul de la moyenne
    mean_values = np.full(len(grid), np.nan, dtype=np.float32)
    valid_cells = count_values > 0
    mean_values[valid_cells] = sum_values[valid_cells] / count_values[valid_cells]
    grid["mean_value"] = mean_values

    # Export en raster
    raster_size_x = len(np.arange(lon_min, lon_max, resolution_x))
    raster_size_y = len(np.arange(lat_min, lat_max, resolution_y))
    transform = from_origin(lon_min, lat_max, resolution_x, resolution_y)

    raster_data = np.full((raster_size_y, raster_size_x), np.nan, dtype=np.float32)
    for idx, row in grid.iterrows():
        poly = row.geometry
        x_idx = int((poly.centroid.x - lon_min) / resolution_x)
        y_idx = int((lat_max - poly.centroid.y) / resolution_y)
        if 0 <= x_idx < raster_size_x and 0 <= y_idx < raster_size_y:
            raster_data[y_idx, x_idx] = row["mean_value"]

    # Stockage dans output_rasters/{polluant}
    output_dir = os.path.join(base_dir, pollutant, "output_rasters")
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, output_name)

    with rasterio.open(output_path, 'w', driver='GTiff', height=raster_size_y, width=raster_size_x,
                       count=1, dtype='float32', crs='EPSG:4326', transform=transform) as dst:
        dst.write(raster_data, 1)

    print(f" Raster enregistré sous {output_path}")

# 🔹 Exemples d'utilisation :

base_dir = "N:/MOD_SERVER/SATELLITES/tropomi_s5_annuel/alternance_moussa/output"
process_netcdf_to_raster("2024", "AER_AI", base_dir, month="03", day="01")  # Raster journalier
#process_netcdf_to_raster("2024", "NO2", base_dir, month="07")            # Raster mensuel
#process_netcdf_to_raster("2024", "NO2", base_dir)                        # Raster annuel


##  **Visualisation**

In [ ]:
from datetime import datetime
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from rasterio.plot import show
from shapely.geometry import box



def visualize_raster(date, polluant="NO2", raster_dir=None, shp_path=None):
    """
    Visualise les rasters journaliers, mensuels ou annuels.

    :param date: Date au format YYYY, YYYYMM ou YYYYMMDD
    :param pollutant: Nom du polluant (ex: "NO2")
    :param base_dir: Répertoire de base contenant les rasters
    :param shp_path: Chemin vers un shapefile pour superposer les limites régionales
    """
    if raster_dir is None:
        raster_dir =  f"N:/MOD_SERVER/SATELLITES/tropomi_s5_annuel/alternance_moussa/output/{polluant}/output_rasters"
    
    # Vérification du polluant
    polluant = polluant.upper()
    if polluant not in POLLUANTS_CONFIG:
        print(f"Polluant '{polluant}' non reconnu.")
        return

    # Déterminer le type de date
    if len(date) == 8:  # Journalier
        year, month, day = date[:4], date[4:6], date[6:8]
        raster_filename = f"{polluant}_mean_{year}_{month}_{day}.tif"
    elif len(date) == 6:  # Mensuel
        year, month = date[:4], date[4:6]
        raster_filename = f"{polluant}_mean_{year}_{month}.tif"
    elif len(date) == 4:  # Annuel
        year = date[:4]
        raster_filename = f"{polluant}_mean_{year}.tif"
    else:
        print("Format de date invalide. Utilisez YYYY, YYYYMM ou YYYYMMDD.")
        return

    # Construire le chemin complet du fichier raster
    raster_path = os.path.join(raster_dir, raster_filename)

    # Vérification de l'existence du fichier raster
    if not os.path.exists(raster_path):
        print(f"Fichier raster introuvable : {raster_path}")
        print("Vérifiez le répertoire ou le format de la date.")
        return

    # Charger le shapefile (facultatif)
    if shp_path and os.path.exists(shp_path):
        shapefile = gpd.read_file(shp_path)
    else:
        shapefile = None

    # Charger le raster
    try:
        with rasterio.open(raster_path) as src:
            raster_data = src.read(1)  # Première bande du raster
            raster_bounds = src.bounds  # Bornes du raster
            extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

            # Masquer les valeurs nodata
            raster_data = np.ma.masked_where(raster_data == src.nodata, raster_data)

    except Exception as e:
        print(f"Erreur lors du chargement du raster : {e}")
        return

    # Visualisation
    fig, ax = plt.subplots(figsize=(12, 8))
    img = ax.imshow(raster_data, extent=extent, cmap="viridis", origin="upper")

    # Ajouter un shapefile (si disponible)
    if shapefile is not None:
        shapefile.boundary.plot(ax=ax, edgecolor="red", linewidth=1, label="Limites régionales")

    # Ajouter une barre de couleur
    unite_name = POLLUANTS_CONFIG[polluant]["unite"]
    cbar = plt.colorbar(img, ax=ax, orientation="vertical", fraction=0.03, pad=0.04)
    cbar.set_label(f"Concentration de {polluant} ({POLLUANTS_CONFIG[polluant]["unite"]}) ")

    # Reformater la date au format jj-mm-aa
    date = datetime.strptime(date, "%Y%m%d").strftime("%d-%m-%Y")

    # Paramètres de l'intrigue
    
    ax.set_title(f"Concentration de {polluant} au {date}")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    if shapefile is not None:
        ax.legend()

    
   # Créer un répertoire "graphes" dans le répertoire du polluant
    graph_dir = f"N:/MOD_SERVER/SATELLITES/tropomi_s5_annuel/alternance_moussa/output/{polluant}/graphes"
    os.makedirs(graph_dir, exist_ok=True)  # Créer le dossier s'il n'existe pas

    # Générer un nom de fichier unique pour le graphique
    graph_filename = f"{polluant}_{date}.png"
    graph_path = os.path.join(graph_dir, graph_filename)

    # Sauvegarder la figure
    plt.savefig(graph_path, dpi=300, bbox_inches="tight")
    print(f"Graphique sauvegardé dans : {graph_path}")
    # Afficher la carte
    plt.show()
    plt.close(fig)  # Fermer la figure pour libérer la mémoire

# Exemple d'utilisation :
# Visualiser un raster journalier

shp_path="N:/MOD_SERVER/SATELLITES/tropomi_s5_annuel/Stage_Moussa/fichiers_shp/contour_region_4326.shp"
base_dir="N:/MOD_SERVER/SATELLITES/tropomi_s5_annuel/alternance_moussa/output"
visualize_raster(date="20240802", polluant="NO2", shp_path=shp_path)
# Visualiser un raster mensuel
#visualize_raster("202407", polluant="NO2",shp_path=shp_path)

# Visualiser un raster annuel
# visualize_raster("2024", pollutant="NO2")
